In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os.path
import requests
from tqdm import tqdm
from bs4 import BeautifulSoup

path = '/content/drive/MyDrive/dataset-flights-brazil-main/dataset-flights-brazil-main/data/extract'

# Flights (ANAC)
url = 'https://www.gov.br/anac/pt-br/assuntos/dados-e-estatisticas/dados-estatisticos'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Get all links
tags_a = soup.find_all('a', {
    'target': '_self',
    'class': 'internal-link',
})
links = [tag_a['href'] for tag_a in tags_a if tag_a['href'].endswith('.csv')]

# Download all files
for link in tqdm(links):
    filename = path + link.split('/')[-1]
    if os.path.exists(filename):
        continue

    file_response = requests.get(link)
    open(filename, 'wb').write(file_response.content)

# Public aerodromes (ANAC)
url = 'https://sistemas.anac.gov.br/dadosabertos/Aerodromos/Lista%20de%20aer%C3%B3dromos%20p%C3%BAblicos/AerodromosPublicos.csv'
file_response = requests.get(url)
open(path + 'public_aerodromes.csv', 'wb').write(file_response.content)

# Airport Codes (DataHub.io)
url = 'https://datahub.io/core/airport-codes/r/airport-codes.csv'
file_response = requests.get(url)
open(path + 'airport_codes.csv', 'wb').write(file_response.content)


100%|██████████| 23/23 [09:14<00:00, 24.13s/it]


6232459

In [ ]:
!pip install geocoder

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 98 kB 2.8 MB/s 


In [ ]:
import glob
import pandas as pd

files = glob.glob('/content/drive/MyDrive/dataset-flights-brazil-main/dataset-flights-brazil-main/data/extractresumo_anual_*.csv')

# Combine all files in the list
combined_csv = pd.concat([pd.read_csv(file, sep=';', encoding='ISO-8859-1') for file in files])

# Rename columns
columns_map = {
    'EMPRESA (SIGLA)': 'company_abbreviation',
    'EMPRESA (NOME)': 'company_name',
    'EMPRESA (NACIONALIDADE)': 'company_nationality',
    'ANO': 'year',
    'MÊS': 'month',
    'AEROPORTO DE ORIGEM (SIGLA)': 'origin_airport_abbreviation',
    'AEROPORTO DE ORIGEM (NOME)': 'origin_airport_name',
    'AEROPORTO DE ORIGEM (UF)': 'origin_airport_state',
    'AEROPORTO DE ORIGEM (REGIÃO)': 'origin_airport_region',
    'AEROPORTO DE ORIGEM (PAÍS)': 'origin_airport_country',
    'AEROPORTO DE ORIGEM (CONTINENTE)': 'origin_airport_continent',
    'AEROPORTO DE DESTINO (SIGLA)': 'destination_airport_abbreviation',
    'AEROPORTO DE DESTINO (NOME)': 'destination_airport_name',
    'AEROPORTO DE DESTINO (UF)': 'destination_airport_state',
    'AEROPORTO DE DESTINO (REGIÃO)': 'destination_airport_region',
    'AEROPORTO DE DESTINO (PAÍS)': 'destination_airport_country',
    'AEROPORTO DE DESTINO (CONTINENTE)': 'destination_airport_continent',
    'NATUREZA': 'nature',
    'GRUPO DE VOO': 'flight_group',
    'PASSAGEIROS PAGOS': 'paid_passenger',
    'PASSAGEIROS GRÁTIS': 'free_passenger',
    'CARGA PAGA (KG)': 'charge_paid_kg',
    'CARGA GRÁTIS (KG)': 'charge_free_kg',
    'CORREIO (KG)': 'mail_kg',
    'ASK': 'ask',
    'RPK': 'rpk',
    'ATK': 'atk',
    'RTK': 'rtk',
    'COMBUSTÍVEL (LITROS)': 'fuel_l',
    'DISTÂNCIA VOADA (KM)': 'distance_flown_km',
    'DECOLAGENS': 'takeoffs',
    'CARGA PAGA KM': 'charge_paid_km',
    'CARGA GRATIS KM': 'charge_free_km',
    'CORREIO KM': 'mail_km',
    'ASSENTOS': 'seats',
    'PAYLOAD': 'payload',
    'HORAS VOADAS': 'fly_hours',
    'BAGAGEM (KG)': 'baggage_kg',
}
combined_csv = combined_csv.rename(columns=columns_map)

# Export to csv
combined_csv.to_csv('/content/drive/MyDrive/dataset-flights-brazil-main/dataset-flights-brazil-main/data/anac.csv', index=False, encoding='utf-8')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DtypeWarning: Columns (24,25,29) have mixed types.Specify dtype option on import or set low_memory=False.
  import sys


In [ ]:
import geocoder
import pandas as pd

path = '/content/drive/MyDrive/dataset-flights-brazil-main/dataset-flights-brazil-main/data/extract'

# ANAC dataset
anac_columns = [
    'origin_airport_abbreviation',
    'origin_airport_name',
    'origin_airport_state',
    'origin_airport_region',
    'origin_airport_country',
    'origin_airport_continent',
    'destination_airport_abbreviation',
    'destination_airport_name',
    'destination_airport_state',
    'destination_airport_region',
    'destination_airport_country',
    'destination_airport_continent',
]
df_anac = pd.read_csv('/content/drive/MyDrive/dataset-flights-brazil-main/dataset-flights-brazil-main/data/anac.csv', usecols=anac_columns)

# Get airports from ANAC
same_columns = {
    'origin_airport_abbreviation': 'code',
    'origin_airport_name': 'name',
    'origin_airport_state': 'state',
    'origin_airport_region': 'region',
    'origin_airport_country': 'country',
    'origin_airport_continent': 'continent',
    'destination_airport_abbreviation': 'code',
    'destination_airport_name': 'name',
    'destination_airport_state': 'state',
    'destination_airport_region': 'region',
    'destination_airport_country': 'country',
    'destination_airport_continent': 'continent',
}
origin_columns = [
    'origin_airport_abbreviation',
    'origin_airport_name',
    'origin_airport_state',
    'origin_airport_region',
    'origin_airport_country',
    'origin_airport_continent',
]
destination_columns = [
    'destination_airport_abbreviation',
    'destination_airport_name',
    'destination_airport_state',
    'destination_airport_region',
    'destination_airport_country',
    'destination_airport_continent',
]

df_origins = df_anac[origin_columns]
df_origins = df_origins.rename(columns=same_columns)

df_destinations = df_anac[destination_columns]
df_destinations = df_destinations.rename(columns=same_columns)

df_airports = pd.concat([df_origins, df_destinations])
df_airports = df_airports.drop_duplicates()

# Airfields dataset
airfields_columns = [
    'Código OACI',
    'LATGEOPOINT',
    'LONGEOPOINT',
]
df_airfields = pd.read_csv(path + 'public_aerodromes.csv',
                           sep=';',
                           usecols=airfields_columns,
                           skiprows=1,
                           encoding='ISO-8859-1')

# Rename columns
columns_map = {
    'CÓDIGO OACI': 'code',
    'LATGEOPOINT': 'lat_geo_point',
    'LONGEOPOINT': 'lon_geo_point',
}
df_airfields = df_airfields.rename(columns=columns_map)

# Others airports
others_airports_columns = [
    'ident',
    'coordinates',
]
df_others_airports = pd.read_csv(path + 'airport_codes.csv', usecols=others_airports_columns)

# Rename columns
columns_map = {
    'ident': 'code',
}
df_others_airports = df_others_airports.rename(columns=columns_map)
df_others_airports[['lon_geo_point', 'lat_geo_point']] = df_others_airports.coordinates.str.split(', ', expand=True, )
df_others_airports = df_others_airports.drop(columns=['coordinates'])

# Coordinates dataframe
df_coordinates = pd.concat([df_airfields, df_others_airports])

# Merge dataframe
df_merge = pd.merge(
    df_airports,
    df_coordinates,
    how='left',
    left_on='code',
    right_on='code')
df_merge = df_merge.dropna(how='all')
df_merge = df_merge.drop_duplicates('code')

# Find the others geolocations
columns = [
    'name',
    'state',
    'region',
    'country',
    'continent',
]
airports_to_find = []
for index, row in df_merge[df_merge.lat_geo_point.isnull()].iterrows():
    label = ''

    for column in columns:
        if type(row[column]) == str and row[column]:
            label += row[column] + ', '

    if not label:
        continue

    geo_result = geocoder.arcgis(label[:-2])
    df_merge.loc[df_merge.code == row['code'], 'lat_geo_point'] = geo_result.latlng[0]
    df_merge.loc[df_merge.code == row['code'], 'lon_geo_point'] = geo_result.latlng[1]

# Save result
df_merge.to_csv('/content/drive/MyDrive/dataset-flights-brazil-main/dataset-flights-brazil-main/data/airports.csv', index=False, encoding='utf-8')

In [ ]:
import networkx as nx
import pandas as pd

df_airports = pd.read_csv('/content/drive/MyDrive/dataset-flights-brazil-main/dataset-flights-brazil-main/data/airports.csv')
df_flights = pd.read_csv('/content/drive/MyDrive/dataset-flights-brazil-main/dataset-flights-brazil-main/data/anac.csv')

# Create graph
G = nx.Graph()

# Add nodes
for index, row in df_airports.iterrows():
    G.add_node(row['code'],
               name=row['name'],
               country=row['country'],
               latitude=row['lat_geo_point'],
               longitude=row['lon_geo_point'],
               region=row['region']
               )

# Add edges
df_edges = df_flights[[
    'origin_airport_abbreviation',
    'destination_airport_abbreviation',
]].dropna()
df_edges = df_edges.groupby(df_edges.columns.tolist(), as_index=False).size()
for index, row in df_edges.iterrows():
    if row['origin_airport_abbreviation'] == row['destination_airport_abbreviation']:
        continue
    G.add_edge(row['origin_airport_abbreviation'], row['destination_airport_abbreviation'], flight_count=row['size'])

# Export to graphml
nx.write_graphml(G, '/content/drive/MyDrive/dataset-flights-brazil-main/dataset-flights-brazil-main/data/air_traffic.graphml')


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (24,25,29) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
